<a href="https://colab.research.google.com/github/LordLean/Acquired-Intelligence-Adaptive-Behaviour/blob/master/AIAB_Labs/ML_Coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"alexholder","key":"d2922de94720de89ce73474159860152"}'}

In [2]:
# Make directory and copy.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

#!pip install kaggle

# Change access permissions.
!chmod 600 /root/.kaggle/kaggle.json

#!kaggle datasets list

# Download using Kaggle API.
!kaggle competitions download -c brighton-a-memorable-city

# Unzip data
!unzip additional_training.csv
!unzip training.csv
!unzip testing.csv

# List files in dir.
!ls

  0% 0.00/19.6k [00:00<?, ?B/s]
100% 19.6k/19.6k [00:00<00:00, 7.91MB/s]
  0% 0.00/1.65M [00:00<?, ?B/s]
100% 1.65M/1.65M [00:00<00:00, 112MB/s]
  0% 0.00/30.0 [00:00<?, ?B/s]
100% 30.0/30.0 [00:00<00:00, 26.6kB/s]
 92% 79.0M/85.7M [00:01<00:00, 42.6MB/s]
100% 85.7M/85.7M [00:01<00:00, 57.3MB/s]
  0% 0.00/81.9k [00:00<?, ?B/s]
100% 81.9k/81.9k [00:00<00:00, 83.2MB/s]
 35% 5.00M/14.2M [00:00<00:01, 8.43MB/s]
100% 14.2M/14.2M [00:00<00:00, 22.3MB/s]
Archive:  additional_training.csv.zip
  inflating: additional_training.csv  
Archive:  training.csv.zip
  inflating: training.csv            
Archive:  testing.csv.zip
  inflating: testing.csv             
additional_training.csv      sample_data		  test_proportions.csv
additional_training.csv.zip  sample_valid_submission.csv  training.csv
annotation_confidence.csv    testing.csv		  training.csv.zip
kaggle.json		     testing.csv.zip


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.neural_network import MLPClassifier

In [0]:
train = pd.read_csv("/content/training.csv")
test = pd.read_csv("/content/testing.csv")

In [5]:
print(test[:].head())
print(test[:].tail())

   ID     CNNs   CNNs.1   CNNs.2  ...  GIST.508  GIST.509  GIST.510  GIST.511
0   1  0.17006  0.56194  0.00000  ...  0.046790  0.045160  0.024613  0.017370
1   2  0.00000  0.00000  0.00000  ...  0.021820  0.021052  0.041294  0.021800
2   3  0.00000  0.00000  0.00000  ...  0.032854  0.036091  0.024968  0.019759
3   4  0.00000  0.00000  0.11766  ...  0.063078  0.033617  0.038866  0.050231
4   5  0.00000  0.00000  0.00000  ...  0.017562  0.011259  0.028914  0.018655

[5 rows x 4609 columns]
          ID     CNNs   CNNs.1   CNNs.2  ...  GIST.508  GIST.509  GIST.510  GIST.511
11869  11870  0.40844  0.00000  0.19764  ...  0.024837  0.056891  0.035763  0.012013
11870  11871  1.05970  0.00000  0.00000  ...  0.019954  0.021078  0.033262  0.028035
11871  11872  0.00000  0.00000  0.00000  ...  0.032306  0.016357  0.022982  0.034376
11872  11873  0.00000  0.00000  0.00000  ...  0.003394  0.005886  0.005672  0.003684
11873  11874  0.54513  0.41296  0.33441  ...  0.039496  0.028001  0.033201  0.0350

In [6]:
print(train[:].head())
print(train[:].tail())

   ID     CNNs   CNNs.1  CNNs.2  ...  GIST.509  GIST.510  GIST.511  prediction
0   1  0.00000  0.00000     0.0  ...  0.016819  0.013863  0.020987           1
1   2  0.00000  0.00000     0.0  ...  0.022172  0.025843  0.020629           1
2   3  0.00000  0.00000     0.0  ...  0.016838  0.018631  0.014774           1
3   4  0.21168  0.53031     0.0  ...  0.022226  0.041484  0.016790           1
4   5  0.00000  0.00000     0.0  ...  0.006265  0.006707  0.013045           1

[5 rows x 4610 columns]
      ID     CNNs   CNNs.1   CNNs.2  ...  GIST.509  GIST.510  GIST.511  prediction
242  243  0.00000  0.00000  0.00000  ...  0.045320  0.029119  0.022450           1
243  244  0.32344  0.36752  0.41601  ...  0.017803  0.033699  0.021059           1
244  245  0.00000  0.00000  0.00000  ...  0.042757  0.077547  0.029385           1
245  246  0.61978  0.00000  0.00000  ...  0.033392  0.048006  0.030306           1
246  247  0.00000  0.00000  0.00000  ...  0.027922  0.022340  0.010484           1

[5

In [7]:
split_spot = 200

train_split_train = train.iloc[:split_spot, 1:-1].to_numpy(copy=True)
train_pred_train = train.iloc[:split_spot, -1].to_numpy(copy=True)
print(len(train_split_train))
print(len(train_pred_train))


train_split_test = train.iloc[split_spot: ,1:-1].to_numpy(copy=True)
train_pred_test = train.iloc[split_spot: ,-1].to_numpy(copy=True)
len(train_pred_test)

200
200


47

In [8]:
X = train_split_train
y = train_pred_train 

clf = MLPClassifier(solver="lbfgs", alpha=1e-5, random_state=1)

clf.fit(X,y)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [9]:
predictions = clf.predict(train_split_test)

accuracy = 0 

for i, (p1,p2) in enumerate(zip(predictions, train_pred_test)):
  if p1 == p2:
    accuracy += 1

print(accuracy / len(predictions))

0.7446808510638298


In [0]:
# Train on total data.

# Get all training values.
train_values = train.iloc[:, 1:-1].to_numpy(copy=True)
# Get all observed predictions.
train_observed = train.iloc[:, -1].to_numpy(copy=True)
# Compare to check equal length.
print(len(train_values))
print(len(train_observed))

X = train_values
y = train_observed 

clf = MLPClassifier(solver="lbfgs", hidden_layer_sizes=(4609,4609,4609,4609), max_iter=500, alpha=1e-5, random_state=1)

clf.fit(X,y)

247
247


In [0]:
# Values to test on.
test_values = test.iloc[:,1:].to_numpy(copy=True)

predictions_final = clf.predict(test_values)

In [0]:
# Create and download .csv file.
rnge = np.arange(1,len(predictions_final)+1)
df = pd.DataFrame({"ID": rnge, "prediction": predictions_final})
df.to_csv("submission1.csv", index=False)

In [0]:
!ls

In [0]:
#files.download("submission1.csv")

In [0]:
# Run to submit to Kaggle competition.
!kaggle competitions submit brighton-a-memorable-city -f submission1.csv -m "woo"